In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import numpy as np
import statsmodels.api as sm
import hockey_scraper
import pickle
import time
import random
pd.set_option('display.max_columns', 100)

In [3]:
nst_to_sched = { 'Anaheim Ducks': 'ANA', 
                      'Arizona Coyotes' : 'ARI', 
                      'Boston Bruins': 'BOS', 
                      'Buffalo Sabres':'BUF',
                      'Calgary Flames': 'CGY', 
                      'Carolina Hurricanes': 'CAR', 
                      'Chicago Blackhawks': 'CHI', 
                      'Colorado Avalanche': 'COL',
                     'Columbus Blue Jackets': 'CBJ',
                     'Dallas Stars': 'DAL',
                     'Detroit Red Wings': 'DET',
                     'Edmonton Oilers': 'EDM',
                     'Florida Panthers': 'FLA',
                     'Los Angeles Kings': 'L.A.',
                     'Minnesota Wild': 'MIN',
                     'Montreal Canadiens': 'MTL',
                     'Nashville Predators': 'NSH',
                     'New Jersey Devils': 'N.J.',
                     'New York Islanders': 'NYI',
                     'New York Rangers': 'NYR',
                     'Ottawa Senators': 'OTT',
                     'Philadelphia Flyers': 'PHI',
                     'Pittsburgh Penguins': 'PIT',
                     'San Jose Sharks': 'S.J.',
                     'St. Louis Blues': 'STL',
                     'Tampa Bay Lightning': 'T.B.',
                     'Toronto Maple Leafs': 'TOR',
                     'Vancouver Canucks': 'VAN',
                      'Vegas Golden Knights':'VGK',
                     'Washington Capitals': 'WSH',
                     'Winnipeg Jets': 'WPG'}

In [4]:
goalie_table_teams = { 'ANA': 'ANA', 
                      'ARI' : 'ARI',
                      'BOS': 'BOS', 
                      'BUF':'BUF',
                      'CGY': 'CGY', 
                      'CAR': 'CAR', 
                      'CHI': 'CHI', 
                      'COL': 'COL',
                     'CBJ': 'CBJ',
                     'DAL': 'DAL',
                     'DET': 'DET',
                     'EDM': 'EDM',
                     'FLA': 'FLO',
                     'L.A': 'LOS',
                     'MIN': 'MIN',
                     'MTL': 'MON',
                     'NSH': 'NSH',
                     'N.J': 'NJD',
                     'NYI': 'NYI',
                     'NYR': 'NYR',
                     'OTT': 'OTT',
                     'PHI': 'PHI',
                     'PIT': 'PIT',
                     'S.J': 'SJS',
                     'STL': 'STL',
                     'T.B': 'TBL',
                     'TOR': 'TOR',
                     'VAN': 'VAN',
                     'WSH': 'WSH',
                     'WPG': 'WPG',
                     'VGK':'VGK'}

In [5]:
#test if i should switch to score and venu adjusted?
#sit = sva
#test 5,10,15 rolling as well
def get_and_format_nst_team_stats(season, sit, rate):
    nst_to_sched = {'Anaheim Ducks': 'ANA',
                     'Arizona Coyotes': 'ARI',
                     'Boston Bruins': 'BOS',
                     'Buffalo Sabres': 'BUF',
                     'Calgary Flames': 'CGY',
                     'Carolina Hurricanes': 'CAR',
                     'Chicago Blackhawks': 'CHI',
                     'Colorado Avalanche': 'COL',
                     'Columbus Blue Jackets': 'CBJ',
                     'Dallas Stars': 'DAL',
                     'Detroit Red Wings': 'DET',
                     'Edmonton Oilers': 'EDM',
                     'Florida Panthers': 'FLA',
                     'Los Angeles Kings': 'L.A',
                     'Minnesota Wild': 'MIN',
                     'Montreal Canadiens': 'MTL',
                     'Nashville Predators': 'NSH',
                     'New Jersey Devils': 'N.J',
                     'New York Islanders': 'NYI',
                     'New York Rangers': 'NYR',
                     'Ottawa Senators': 'OTT',
                     'Philadelphia Flyers': 'PHI',
                     'Pittsburgh Penguins': 'PIT',
                     'San Jose Sharks': 'S.J',
                     'St Louis Blues': 'STL',
                     'Tampa Bay Lightning': 'T.B',
                     'Toronto Maple Leafs': 'TOR',
                     'Vancouver Canucks': 'VAN',
                     'Vegas Golden Knights': 'VGK',
                     'Washington Capitals': 'WSH',
                     'Winnipeg Jets': 'WPG'}
    
    url = 'https://www.naturalstattrick.com/games.php?fromseason={}&thruseason={}&stype=2&sit={}&loc=B&team=All&rate={}'.format(
        season,
        season,
        sit,
        rate)
    df = pd.read_html(url, header=0, index_col = 0, na_values=["-"])[0]
    df.reset_index(inplace = True)
    df['Date'] = df['Game'].apply(lambda x: pd.to_datetime(x[0:10]))
    df['Game_Number'] = df.groupby('Team').cumcount() + 1
    #rename Team_Date to team key or something like that
    df = df.replace({'Team': nst_to_sched})

    df['Team_Key'] = df['Team'].astype(str)+'_'+df['Date'].astype(str)
    return df

In [6]:
def merge_team_stats(primary_df, pp_df, pk_df):
    primary_df = primary_df.merge(pk_df[['Team_Key', 'TOI', 'xGA']], on = 'Team_Key', how = 'left', suffixes = ('','_pk') )
    primary_df = primary_df.merge(pp_df[['Team_Key', 'TOI', 'xGF']], on = 'Team_Key', how = 'left', suffixes = ('','_pp') )
    return primary_df

In [395]:
def calculate_team_features(df, rolling_games = 20):
    rolling_games, rolling_games 
    df['sum_rolling20_TOI_5v5'] = df.groupby('Team')['TOI'].transform(lambda x: x.rolling(rolling_games, rolling_games).sum().shift())
    df['sum_rolling20_FF_5v5'] = df.groupby('Team')['FF'].transform(lambda x: x.rolling(rolling_games, rolling_games ).sum().shift())
    df['sum_rolling20_FA_5v5'] = df.groupby('Team')['FA'].transform(lambda x: x.rolling(rolling_games, rolling_games ).sum().shift())
    df['sum_rolling20_GF_5v5'] = df.groupby('Team')['GF'].transform(lambda x: x.rolling(rolling_games, rolling_games ).sum().shift())
    df['sum_rolling20_GA_5v5'] = df.groupby('Team')['GA'].transform(lambda x: x.rolling(rolling_games, rolling_games ).sum().shift())
    df['sum_rolling20_xGF_5v5'] = df.groupby('Team')['xGF'].transform(lambda x: x.rolling(rolling_games, rolling_games ).sum().shift())
    df['sum_rolling20_xGA_5v5'] = df.groupby('Team')['xGA'].transform(lambda x: x.rolling(rolling_games, rolling_games ).sum().shift())
    df['sum_rolling20_SF_5v5'] = df.groupby('Team')['SF'].transform(lambda x: x.rolling(rolling_games, rolling_games ).sum().shift())
    df['last_20_FF%_5v5'] = df['sum_rolling20_FF_5v5']*100/ (df['sum_rolling20_FF_5v5']+df['sum_rolling20_FA_5v5'])
    df['last_20_GF%_5v5'] = df['sum_rolling20_GF_5v5']*100/ (df['sum_rolling20_GF_5v5']+df['sum_rolling20_GA_5v5'])
    df['last_20_xGF%_5v5'] = df['sum_rolling20_xGF_5v5']*100/ (df['sum_rolling20_xGF_5v5']+df['sum_rolling20_xGA_5v5'])
    df['last_20_SH%'] = df['sum_rolling20_GF_5v5']*100 / df['sum_rolling20_SF_5v5']
    
    
    #fix NaNs in pp and pk features
    df['TOI_pp'] = np.where(df['TOI_pp'].isna(), 0, df['TOI_pp'])
    df['TOI_pk'] = np.where(df['TOI_pk'].isna(), 0, df['TOI_pk'])
    df['xGF_pp'] = np.where(df['xGF_pp'].isna(), 0, df['xGF_pp'])
    df['xGA_pk'] = np.where(df['xGA_pk'].isna(), 0, df['xGA_pk'])
    
    #pp features
    df['sum_rolling20_TOI_pp'] = df.groupby('Team')['TOI_pp'].transform(lambda x: x.rolling(rolling_games, rolling_games ).sum().shift())
    df['sum_rolling20_xGF_pp'] = df.groupby('Team')['xGF_pp'].transform(lambda x: x.rolling(rolling_games, rolling_games ).sum().shift())
    df['last20_pp_TOI_per_game'] = df.groupby('Team')['TOI_pp'].transform(lambda x: x.rolling(rolling_games, rolling_games ).mean().shift())
    df['last20_xGF_per_min_pp'] = df['sum_rolling20_xGF_pp'] / df['sum_rolling20_TOI_pp'] 
    
    #pk features
    df['sum_rolling20_TOI_pk'] = df.groupby('Team')['TOI_pk'].transform(lambda x: x.rolling(rolling_games, rolling_games ).sum().shift())
    df['sum_rolling20_xGA_pk'] = df.groupby('Team')['xGA_pk'].transform(lambda x: x.rolling(rolling_games, rolling_games ).sum().shift())
    df['last20_pk_TOI_per_game'] = df.groupby('Team')['TOI_pk'].transform(lambda x: x.rolling(rolling_games, rolling_games ).mean().shift())
    df['last20_xGA_per_min_pk'] = df['sum_rolling20_xGA_pk'] / df['sum_rolling20_TOI_pk'] 
    
    #to get back to back category
    df['Last_Game_Date'] = df.groupby('Team')['Date'].shift()
    df['Days_Since_Last_Game'] = df['Date'] - df['Last_Game_Date']
    df['B2B'] = np.where(df['Days_Since_Last_Game'] == '1 days', 1, 0)
    
    return df

In [8]:
#get starters
def get_starters(year):
    goalie_table_teams = { 'ANA': 'ANA', 
                      'ARI' : 'ARI',
                      'BOS': 'BOS', 
                      'BUF':'BUF',
                      'CGY': 'CGY', 
                      'CAR': 'CAR', 
                      'CHI': 'CHI', 
                      'COL': 'COL',
                     'CBJ': 'CBJ',
                     'DAL': 'DAL',
                     'DET': 'DET',
                     'EDM': 'EDM',
                     'FLA': 'FLO',
                     'L.A': 'LOS',
                     'MIN': 'MIN',
                     'MTL': 'MON',
                     'NSH': 'NSH',
                     'N.J': 'NJD',
                     'NYI': 'NYI',
                     'NYR': 'NYR',
                     'OTT': 'OTT',
                     'PHI': 'PHI',
                     'PIT': 'PIT',
                     'S.J': 'SJS',
                     'STL': 'STL',
                     'T.B': 'TBL',
                     'TOR': 'TOR',
                     'VAN': 'VAN',
                     'WSH': 'WSH',
                     'WPG': 'WPG',
                     'VGK':'VGK'}
    counter = 0
    for k,v in goalie_table_teams.items():
        print(k)
        starter_url = 'http://hockeygoalies.org/bio/nhl/logs/{}{}.html'.format(v, year)
        goalies = pd.read_html(starter_url)[0]
        goalies.replace(to_replace=['(BU)', np.NaN], value = 'DNP', inplace = True)
        goalies.drop(columns = ['DEC'], inplace = True)
        goalies.drop(index  = goalies.iloc[-1].name, inplace = True)
        goalies['starter'] = 'placeholder'

        starter = []
        for i, row in goalies.iterrows():
            for n in range(len(row)):
                if row[n][0] == 'W' or row[n][0] == 'L':
                    starter.append(goalies.columns[n])

        goalies['starter'] = starter
        goalies['Team'] = k
        goalies['DATE'] = pd.to_datetime(goalies['DATE'])
        goalies['DATE'] = pd.to_datetime(goalies['DATE'])
        goalies['Team_Key'] = goalies['Team'].astype(str)+'_'+goalies['DATE'].astype(str)
        columns = ['Team','DATE', 'OPPONENT', 'starter', 'Team_Key']
        if counter == 0:
            master = goalies
        if counter != 0:
            master = pd.concat( [master[columns], goalies[columns]])
        counter +=1
    return master
                

In [9]:
#'2017-10-04' to '2018-04-08'
def get_game_results(season_start, season_end):
    sched_df = hockey_scraper.scrape_schedule(season_start, season_end)
    sched_df['Home_Team_Won'] = np.where(sched_df['home_score'] > sched_df['away_score'], 1, 0)
    sched_df['Home_Team_Key'] = sched_df['home_team'].astype(str)+'_'+sched_df['date'].astype(str)
    sched_df['Away_Team_Key'] = sched_df['away_team'].astype(str)+'_'+sched_df['date'].astype(str)
    return sched_df

In [322]:
def merge_starters_and_features(game_results_df, goalies_df, features_df, feature_columns, goalie_feature_columns):
    goalies_df = goalies_df[goalies_df['TOI'] >=28]
    df = game_results_df.merge(goalies_df[goalie_feature_columns].add_prefix('home_'), left_on = 'Home_Team_Key', right_on = 'home_Team_Key', how = 'left').rename(columns ={'home_Name':'home_goalie'}).drop(columns = 'home_Team_Key')
    df = df.merge(goalies_df[goalie_feature_columns].add_prefix('away_'), left_on = 'Away_Team_Key', right_on = 'away_Team_Key', how = 'left').rename(columns ={'away_Name':'away_goalie'}).drop(columns = 'away_Team_Key')
    df = df.merge(features_df[feature_columns].add_prefix('home_'), left_on = 'Home_Team_Key', right_on = 'home_Team_Key', how = 'left')
    df = df.merge(features_df[feature_columns].add_prefix('away_'), left_on = 'Away_Team_Key', right_on = 'away_Team_Key', how = 'left')
    return df

In [235]:
feature_columns = ['Team_Key', 'last_20_FF%_5v5', 'last_20_GF%_5v5', 'last_20_xGF%_5v5', 'last_20_SH%', 'last20_pp_TOI_per_game', 'last20_xGF_per_min_pp','last20_pk_TOI_per_game', 'last20_xGA_per_min_pk', 'B2B']
goalie_feature_columns = ['Team_Key', 'Name', 'Last_20_FenwickSV%', 'Last_20_GSAx/60', 'Last_20_HDCSV%']

### Get Goalie Data

In [21]:
#import dictionary with goalie names and IDs from NHL API
infile = open("data/goalie_ids.pickle",'rb')
goalie_ids = pickle.load(infile)
infile.close()

In [141]:
## scrape season long stats to get name of all goalies who played in time frame
goalie_list = pd.read_html('https://www.naturalstattrick.com/playerteams.php?fromseason=20182019&thruseason=20202021&stype=2&sit=5v5&score=all&stdoi=g&rate=n&team=ALL&pos=S&loc=B&toi=0&gpfilt=none&fd=&td=&tgp=410&lines=single&draftteam=ALL')[0]

In [143]:
#find which goalies are missing from goalie_ids dictionary
missing_goalies2 = [g for g in list(goalie_list['Player']) if g not in goalie_ids.keys() ]

In [146]:
## Cal Petersen already in Data Dictionary as Calvin Petersen 
missing_goalies2.remove('Cal Petersen')

In [147]:
missing_goalies2

['Richard Bachman',
 'Edward Pasquale',
 'Garret Sparks',
 'Antoine Bibeau',
 'Pheonix Copley',
 'Dan Vladar',
 'Landon Bow',
 'David Ayres',
 'Kevin Boyle',
 'Stuart Skinner',
 'Hunter Miska',
 'Matiss Kivlenieks',
 'Gilles Senn',
 'Jeremy Swayman',
 'Logan Thompson',
 'Kaden Fulcher',
 'Veini Vehvilainen',
 'Ivan Prosvetov',
 'Alexei Melnichuk']

In [145]:
goalie_ids['Calvin Petersen']

8477361

In [148]:
#manually looked up missing ids. Will write code in future to grab any missing active IDs
missing_id = ['8473614', '8475277', '8476343', '8477312', '8477831', '8478435', '8479016', '8479188', '8479294', '8479973', '8480112',  '8480162', '8480213', '8480280',  '8480313', '8480363',  '8481001', '8481031', '8482246']

In [149]:
for i,g in enumerate(missing_goalies2[:len(missing_id)]):
    goalie_ids[g] = missing_id[i]

In [233]:
def goalie_features(df, rolling_games = 20):
    rolling_games = rolling_games

    df['Date'] = df['Game'].apply(lambda x: pd.to_datetime(x[0:10]))
    df['Team_Key'] = df['Team'].astype(str)+'_'+df['Date'].astype(str)
    
    df['Last_20_TOI'] = df.groupby('ID')['TOI'].transform(lambda x: x.rolling(rolling_games, rolling_games).sum().shift())
    df['Last_20_FA'] = df.groupby('ID')['FA'].transform(lambda x: x.rolling(rolling_games, rolling_games).sum().shift())
    df['Last_20_SA'] = df.groupby('ID')['SA'].transform(lambda x: x.rolling(rolling_games, rolling_games).sum().shift())
    df['Last_20_GA'] = df.groupby('ID')['GA'].transform(lambda x: x.rolling(rolling_games, rolling_games).sum().shift())
    df['Last_20_xGA'] = df.groupby('ID')['xGA'].transform(lambda x: x.rolling(rolling_games, rolling_games).sum().shift())
    df['Last_20_HDCA'] = df.groupby('ID')['HDCA'].transform(lambda x: x.rolling(rolling_games, rolling_games).sum().shift())
    df['Last_20_HDGA'] = df.groupby('ID')['HDGA'].transform(lambda x: x.rolling(rolling_games, rolling_games).sum().shift())
    
    df['Last_20_FenwickSV%'] =  (df['Last_20_FA'] - df['Last_20_GA']) /  df['Last_20_FA']
    df['Last_20_GSAx'] = df['Last_20_xGA'] - df['Last_20_GA']
    df['Last_20_GSAx/60'] =  df['Last_20_GSAx']*60 /  df['Last_20_TOI']
    df['Last_20_HDCSV%'] = (df['Last_20_HDCA'] - df['Last_20_HDGA'] ) / df['Last_20_HDCA'] 
    return df

In [289]:
# only use at most 2 seasons
def get_goalie_data(goalie_ids, start_year, end_year):
    counter = 0
    for name, gid in goalie_ids.items():

        sequence = [x/10 for x in range(60, 120)]
        time.sleep(random.choice(sequence))
        url = 'https://www.naturalstattrick.com/playerreport.php?fromseason={}&thruseason={}&playerid={}&sit=all&stype=2&stdoi=oi&rate=n&v=g'.format(start_year, end_year, gid)
        #due to number of http requests, NST may ban your IP before the loop finishes. I needed to use a VPN to get around this. If IP gets banned, this function will still return the current DF and you can call the function again and pass in an updated goalie dictionary to get the rest
        try:
            individual_df = pd.read_html(url)[0]
            individual_df['Name'] = name
            individual_df['ID'] = gid
        except:
            print(f'Ended before {name}')
            return all_goalies4 

        if counter == 0:
            all_goalies4 = individual_df
            print(name)
            print(counter)
        elif counter != 0:
            all_goalies4 = pd.concat([all_goalies4, individual_df])
            print(name)
            print(counter)


        counter +=1
    
    return all_goalies4

In [278]:
goalies_161718 = get_goalie_data(goalie_ids, 20162017, 20172018)

Scott Wedgewood
0
Aaron Dell
1
Mackenzie Blackwood
2
Cory Schneider
3
Semyon Varlamov
4
Ilya Sorokin
5
Keith Kinkaid
6
Igor Shesterkin
7
Alexandar Georgiev
8
Brian Elliott
9
Alex Lyon
10
Carter Hart
11
Emil Larmi
12
Tristan Jarry
13
Casey DeSmith
14
Jaroslav Halak
15
Tuukka Rask
16
Michael Houser
17
Ukko-Pekka Luukkonen
18
Dustin Tokarski
19
Carter Hutton
20
Linus Ullmark
21
Charlie Lindgren
22
Carey Price
23
Jake Allen
24
Cayden Primeau
25
Joey Daccord
26
Anton Forsberg
27
Matt Murray
28
Marcus Hogberg
29
Filip Gustavsson
30
Frederik Andersen
31
Jack Campbell
32
David Rittich
33
James Reimer
34
Petr Mrazek
35
Alex Nedeljkovic
36
Philippe Desrosiers
37
Sam Montembeault
38
Sergei Bobrovsky
39
Chris Driedger
40
Spencer Knight
41
Christopher Gibson
42
Curtis McElhinney
43
Andrei Vasilevskiy
44
Craig Anderson
45
Vitek Vanecek
46
Ilya Samsonov
47
Malcolm Subban
48
Collin Delia
49
Kevin Lankinen
50
Thomas Greiss
51
Jonathan Bernier
52
Kasimir Kaskisuo
53
Pekka Rinne
54
Juuse Saros
55
Jordan 

In [283]:
missing1617 = [g for g in goalie_ids.keys() if g not in list(goalies_161718['Name'])]

In [284]:
len(missing1617)

83

In [287]:
missing1617_dict = {k:v for k,v in goalie_ids.items() if k in missing1617}

In [291]:
goalies_161718b = get_goalie_data(missing1617_dict, 20162017, 20172018)

Mackenzie Blackwood
0
Ilya Sorokin
1
Igor Shesterkin
2
Carter Hart
3
Emil Larmi
4
Michael Houser
5
Ukko-Pekka Luukkonen
6
Cayden Primeau
7
Joey Daccord
8
Marcus Hogberg
9
Filip Gustavsson
10
Philippe Desrosiers
11
Sam Montembeault
12
Spencer Knight
13
Vitek Vanecek
14
Ilya Samsonov
15
Kevin Lankinen
16
Kasimir Kaskisuo
17
Jordan Binnington
18
Ville Husso
19
Artyom Zagidulin
20
Pavel Francouz
21
Adam Werner
22
Peyton Jones
23
Jonas Johansson
24
Mikko Koskinen
25
Michael Dipietro
26
Arturs Silovs
27
Jake Oettinger
28
Troy Grosenick
29
Calvin Petersen
30
Josef Korenar
31
Elvis Merzlikins
32
Kaapo Kahkonen
33
Antti Niemi
34
Calvin Pickard
35
Cam Ward
36
Cameron Talbot
37
Chad Johnson
38
Corey Crawford
39
Daniel Taylor
40
Eddie Lack
41
Eric Comrie
42
Harri Sateri
43
Henrik Lundqvist
44
Jared Coreau
45
Jean-Francois Berube
46
Jeff Glass
47
Jimmy Howard
48
Jon Gillies
49
Kari Lehtonen
50
Ken Appleby
51
Maxime Lagace
52
Michael Hutchinson
53
Michal Neuvirth
54
Mike Condon
55
Mike McKenna
56
On

In [292]:
goalies_181920 = get_goalie_data(goalie_ids, 20182019, 20192020)

Scott Wedgewood
0
Aaron Dell
1
Mackenzie Blackwood
2
Cory Schneider
3
Semyon Varlamov
4
Ilya Sorokin
5
Keith Kinkaid
6
Igor Shesterkin
7
Alexandar Georgiev
8
Brian Elliott
9
Alex Lyon
10
Carter Hart
11
Emil Larmi
12
Tristan Jarry
13
Casey DeSmith
14
Jaroslav Halak
15
Tuukka Rask
16
Michael Houser
17
Ukko-Pekka Luukkonen
18
Dustin Tokarski
19
Carter Hutton
20
Linus Ullmark
21
Charlie Lindgren
22
Carey Price
23
Jake Allen
24
Cayden Primeau
25
Joey Daccord
26
Anton Forsberg
27
Matt Murray
28
Marcus Hogberg
29
Filip Gustavsson
30
Frederik Andersen
31
Jack Campbell
32
David Rittich
33
James Reimer
34
Petr Mrazek
35
Alex Nedeljkovic
36
Philippe Desrosiers
37
Sam Montembeault
38
Sergei Bobrovsky
39
Chris Driedger
40
Spencer Knight
41
Christopher Gibson
42
Curtis McElhinney
43
Andrei Vasilevskiy
44
Craig Anderson
45
Vitek Vanecek
46
Ilya Samsonov
47
Malcolm Subban
48
Collin Delia
49
Kevin Lankinen
50
Thomas Greiss
51
Jonathan Bernier
52
Kasimir Kaskisuo
53
Pekka Rinne
54
Juuse Saros
55
Jordan 

In [293]:
goalies_2021 = get_goalie_data(goalie_ids, 20202021, 20202021)

Scott Wedgewood
0
Aaron Dell
1
Mackenzie Blackwood
2
Cory Schneider
3
Semyon Varlamov
4
Ilya Sorokin
5
Keith Kinkaid
6
Igor Shesterkin
7
Alexandar Georgiev
8
Brian Elliott
9
Alex Lyon
10
Carter Hart
11
Emil Larmi
12
Tristan Jarry
13
Casey DeSmith
14
Jaroslav Halak
15
Tuukka Rask
16
Michael Houser
17
Ukko-Pekka Luukkonen
18
Dustin Tokarski
19
Carter Hutton
20
Linus Ullmark
21
Charlie Lindgren
22
Carey Price
23
Jake Allen
24
Cayden Primeau
25
Joey Daccord
26
Anton Forsberg
27
Matt Murray
28
Marcus Hogberg
29
Filip Gustavsson
30
Frederik Andersen
31
Jack Campbell
32
David Rittich
33
James Reimer
34
Petr Mrazek
35
Alex Nedeljkovic
36
Philippe Desrosiers
37
Sam Montembeault
38
Sergei Bobrovsky
39
Chris Driedger
40
Spencer Knight
41
Christopher Gibson
42
Curtis McElhinney
43
Andrei Vasilevskiy
44
Craig Anderson
45
Vitek Vanecek
46
Ilya Samsonov
47
Malcolm Subban
48
Collin Delia
49
Kevin Lankinen
50
Thomas Greiss
51
Jonathan Bernier
52
Kasimir Kaskisuo
53
Pekka Rinne
54
Juuse Saros
55
Jordan 

In [294]:
goalies_all_B = pd.concat([goalies_161718, goalies_161718b, goalies_181920, goalies_2021])

In [295]:
goalie_features_dfB = goalie_features(goalies_all_B)

In [296]:
goalie_features_dfB

,Game,Team,TOI,CF,CA,CF%,FF,FA,FF%,SF,SA,SF%,GF,GA,GF%,xGF,xGA,xGF%,SCF,SCA,SCF%,SCGF,SCGA,SCGF%,HDCF,HDCA,HDCF%,HDGF,HDGA,HDGF%,MDCF,MDCA,MDCF%,MDGF,MDGA,MDGF%,LDCF,LDCA,LDCF%,LDGF,LDGA,LDGF%,On-Ice SH%,On-Ice SV%,PDO,Off. Zone Starts,Neu. Zone Starts,Def. Zone Starts,On The Fly Starts,Off. Zone Start %,Off. Zone Faceoffs,Neu. Zone Faceoffs,Def. Zone Faceoffs,Off. Zone Faceoff %,Name,ID,Date,Team_Key,Last_20_TOI,Last_20_FA,Last_20_SA,Last_20_GA,Last_20_xGA,Last_20_HDCA,Last_20_HDGA,Last_20_FenwickSV%,Last_20_GSAx,Last_20_GSAx/60,Last_20_HDCSV%
0,2017-10-30 ARI at PHI,ARI,64.666667,58,60,49.15,42,48,46.67,33,31,51.56,4,3,57.14,2.98,2.27,56.78,32,21,60.38,3,3,50,13,8,61.9,1,2,33.33,19,13,59.38,2,1,66.67,18,35,33.96,1,0,100.00,12.12,90.32,1.024,1,4,0,0,100.00,24,23,25,48.98,Scott Wedgewood,8475809,2017-10-30,ARI_2017-10-30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2017-10-31 ARI at DET,ARI,58.333333,54,63,46.15,41,53,43.62,32,39,45.07,3,4,42.86,1.73,2.50,41,28,32,46.67,1,2,33.33,11,15,42.31,1,1,50,17,17,50,0,1,0.00,24,29,45.28,2,2,50.00,9.38,89.74,0.991,1,4,0,0,100.00,18,22,24,42.86,Scott Wedgewood,8475809,2017-10-31,ARI_2017-10-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2017-11-06 ARI at WSH,ARI,63.883333,49,75,39.52,38,60,38.78,26,40,39.39,2,3,40,1.81,2.84,38.95,20,31,39.22,2,1,66.67,8,10,44.44,2,1,66.67,12,21,36.36,0,0,-,26,37,41.27,0,2,0.00,7.69,92.5,1.002,1,4,0,0,100.00,14,18,26,35,Scott Wedgewood,8475809,2017-11-06,ARI_2017-11-06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2017-11-14 ARI at WPG,ARI,34.866667,32,26,55.17,20,21,48.78,18,17,51.43,1,1,50,1.15,1.41,45,17,15,53.13,0,1,0,5,7,41.67,0,1,0,12,8,60,0,0,-,14,9,60.87,1,0,100.00,5.56,94.12,0.997,0,2,0,0,-,10,14,7,58.82,Scott Wedgewood,8475809,2017-11-14,ARI_2017-11-14,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2017-11-22 S.J at ARI,ARI,44.333333,32,32,50,23,24,48.94,19,16,54.29,1,2,33.33,1.22,1.41,46.35,12,16,42.86,1,2,33.33,8,7,53.33,1,2,33.33,4,9,30.77,0,0,-,15,15,50,0,0,-,5.26,87.5,0.928,0,3,1,0,0.00,9,13,22,29.03,Scott Wedgewood,8475809,2017-11-22,ARI_2017-11-22,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4,2021-04-16 NYI at BOS,BOS,60.000000,46,44,51.11,35,35,50,28,25,52.83,3,0,100,2.58,2.17,54.29,22,25,46.81,2,0,100,10,8,55.56,2,0,100.00,12,17,41.38,0,0,-,24,17,58.54,1,0,100.00,10.71,100,1.107,0,3,0,0,-,15,13,27,35.71,Jeremy Swayman,8480280,2021-04-16,BOS_2021-04-16,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,2021-04-22 BOS at BUF,BOS,59.983333,55,54,50.46,46,42,52.27,38,30,55.88,5,1,83.33,1.96,2.23,46.77,19,20,48.72,4,1,80,7,11,38.89,1,1,50.00,12,9,57.14,3,0,100.00,34,29,53.97,1,0,100.00,13.16,96.67,1.098,1,3,0,0,100.00,20,18,19,51.28,Jeremy Swayman,8480280,2021-04-22,BOS_2021-04-22,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,2021-04-25 BOS at PIT,BOS,58.166667,43,52,45.26,35,38,47.95,28,29,49.12,0,1,0,1.28,2.07,38.25,13,30,30.23,0,1,0,2,11,15.38,0,0,-,11,19,36.67,0,1,0.00,25,22,53.19,0,0,-,0,96.55,0.966,0,3,1,0,0.00,19,11,24,44.19,Jeremy Swayman,8480280,2021-04-25,BOS_2021-04-25,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,2021-03-10 VGK at MIN,VGK,8.250000,6,5,54.55,6,3,66.67,6,2,75,1,0,100,0.27,0.15,64.31,5,3,62.5,1,0,100,0,1,0,0,0,-,5,2,71.43,1,0,100,1,2,33.33,0,0,-,16.67,100,1.167,0,1,1,2,0,2,4,1,66.67,Logan Thompson,8480313,2021-03-10,VGK_2021-03-10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [298]:
#save to CSV
goalie_features_dfB.to_csv('goalie_game_logs.csv')

### Determine Goalie Stats to Impute for Goalies Having Player Less Than 20 Games

In [347]:
goalie_features_dfB.isna().sum()

Game                     0
Team                     0
TOI                      0
CF                       0
CA                       0
                      ... 
Last_20_HDGA          2246
Last_20_FenwickSV%    2246
Last_20_GSAx          2246
Last_20_GSAx/60       2246
Last_20_HDCSV%        2246
Length: 69, dtype: int64

In [353]:
#set df for goalies who hadnt played 20 games at that point
ig_df = goalie_features_dfB[(goalie_features_dfB['Last_20_FenwickSV%'].isna()) & (goalie_features_dfB['Date'] >= '2017-10-04')]

In [354]:
ig_df.head()

,Game,Team,TOI,CF,CA,CF%,FF,FA,FF%,SF,SA,SF%,GF,GA,GF%,xGF,xGA,xGF%,SCF,SCA,SCF%,SCGF,SCGA,SCGF%,HDCF,HDCA,HDCF%,HDGF,HDGA,HDGF%,MDCF,MDCA,MDCF%,MDGF,MDGA,MDGF%,LDCF,LDCA,LDCF%,LDGF,LDGA,LDGF%,On-Ice SH%,On-Ice SV%,PDO,Off. Zone Starts,Neu. Zone Starts,Def. Zone Starts,On The Fly Starts,Off. Zone Start %,Off. Zone Faceoffs,Neu. Zone Faceoffs,Def. Zone Faceoffs,Off. Zone Faceoff %,Name,ID,Date,Team_Key,Last_20_TOI,Last_20_FA,Last_20_SA,Last_20_GA,Last_20_xGA,Last_20_HDCA,Last_20_HDGA,Last_20_FenwickSV%,Last_20_GSAx,Last_20_GSAx/60,Last_20_HDCSV%
0,2017-10-30 ARI at PHI,ARI,64.666667,58,60,49.15,42,48,46.67,33,31,51.56,4,3,57.14,2.98,2.27,56.78,32,21,60.38,3,3,50,13,8,61.9,1,2,33.33,19,13,59.38,2,1,66.67,18,35,33.96,1,0,100.00,12.12,90.32,1.024,1,4,0,0,100.00,24,23,25,48.98,Scott Wedgewood,8475809,2017-10-30,ARI_2017-10-30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2017-10-31 ARI at DET,ARI,58.333333,54,63,46.15,41,53,43.62,32,39,45.07,3,4,42.86,1.73,2.50,41,28,32,46.67,1,2,33.33,11,15,42.31,1,1,50,17,17,50,0,1,0.00,24,29,45.28,2,2,50.00,9.38,89.74,0.991,1,4,0,0,100.00,18,22,24,42.86,Scott Wedgewood,8475809,2017-10-31,ARI_2017-10-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2017-11-06 ARI at WSH,ARI,63.883333,49,75,39.52,38,60,38.78,26,40,39.39,2,3,40,1.81,2.84,38.95,20,31,39.22,2,1,66.67,8,10,44.44,2,1,66.67,12,21,36.36,0,0,-,26,37,41.27,0,2,0.00,7.69,92.5,1.002,1,4,0,0,100.00,14,18,26,35,Scott Wedgewood,8475809,2017-11-06,ARI_2017-11-06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2017-11-14 ARI at WPG,ARI,34.866667,32,26,55.17,20,21,48.78,18,17,51.43,1,1,50,1.15,1.41,45,17,15,53.13,0,1,0,5,7,41.67,0,1,0,12,8,60,0,0,-,14,9,60.87,1,0,100.00,5.56,94.12,0.997,0,2,0,0,-,10,14,7,58.82,Scott Wedgewood,8475809,2017-11-14,ARI_2017-11-14,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2017-11-22 S.J at ARI,ARI,44.333333,32,32,50,23,24,48.94,19,16,54.29,1,2,33.33,1.22,1.41,46.35,12,16,42.86,1,2,33.33,8,7,53.33,1,2,33.33,4,9,30.77,0,0,-,15,15,50,0,0,-,5.26,87.5,0.928,0,3,1,0,0.00,9,13,22,29.03,Scott Wedgewood,8475809,2017-11-22,ARI_2017-11-22,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [357]:
ig_TOI = ig_df['TOI'].sum()
ig_FA = ig_df['FA'].sum()
ig_GA = ig_df['GA'].sum()
ig_xGA = ig_df['xGA'].sum()
ig_HDCA = ig_df['HDCA'].sum()
ig_HDGA = ig_df['HDGA'].sum()

In [358]:
ig_FenwickSV = (ig_FA - ig_GA) /ig_FA
ig_GSAx = ig_xGA - ig_GA
ig_GSAx60 = (ig_GSAx*60) / ig_TOI
ig_HDCSV = (ig_HDCA - ig_HDGA )/ ig_HDCA

In [375]:
display(ig_FenwickSV)
display(eg_FenwickSV)
display(goalie_features_dfB['Last_20_FenwickSV%'].quantile(.5))
display(goalie_features_dfB['Last_20_FenwickSV%'].mean())

0.934156050955414

0.9347642975398647

0.9359673024523161

0.9357775028284154

In [370]:
goalie_features_dfB['Last_20_FenwickSV%'].std()

0.010027107801083953

In [376]:
display(ig_GSAx60)
display(eg_GSAx60)
display(goalie_features_dfB['Last_20_GSAx/60'].quantile(.5))
display(goalie_features_dfB['Last_20_GSAx/60'].mean())

-0.2802337315202244

-0.230296886503291

-0.18358138942121538

-0.19155023944049981

In [371]:
goalie_features_dfB['Last_20_GSAx/60'].std()

0.42942641496477013

In [377]:
display(ig_HDCSV )
display(eg_HDCSV)
display(goalie_features_dfB['Last_20_HDCSV%'].quantile(.5))
display(goalie_features_dfB['Last_20_HDCSV%'].mean())

0.861424758776432

0.8617668177534443

0.8654708520179372

0.8633200496189025

In [372]:
eg_df = goalie_features_dfB[(~goalie_features_dfB['Last_20_FenwickSV%'].isna()) & (goalie_features_dfB['Date'] >= '2017-10-04')]

In [373]:
eg_df

,Game,Team,TOI,CF,CA,CF%,FF,FA,FF%,SF,SA,SF%,GF,GA,GF%,xGF,xGA,xGF%,SCF,SCA,SCF%,SCGF,SCGA,SCGF%,HDCF,HDCA,HDCF%,HDGF,HDGA,HDGF%,MDCF,MDCA,MDCF%,MDGF,MDGA,MDGF%,LDCF,LDCA,LDCF%,LDGF,LDGA,LDGF%,On-Ice SH%,On-Ice SV%,PDO,Off. Zone Starts,Neu. Zone Starts,Def. Zone Starts,On The Fly Starts,Off. Zone Start %,Off. Zone Faceoffs,Neu. Zone Faceoffs,Def. Zone Faceoffs,Off. Zone Faceoff %,Name,ID,Date,Team_Key,Last_20_TOI,Last_20_FA,Last_20_SA,Last_20_GA,Last_20_xGA,Last_20_HDCA,Last_20_HDGA,Last_20_FenwickSV%,Last_20_GSAx,Last_20_GSAx/60,Last_20_HDCSV%
20,2017-10-07 L.A at S.J,S.J,23.033333,20,19,51.28,15,12,55.56,9,9,50,0,0,-,0.43,0.86,33.52,5,9,35.71,0,0,-,0,3,0,0,0,-,5,6,45.45,0,0,-,11,8,57.89,0,0,-,0,100,1,2,2,0,0,100.00,7,11,6,53.85,Aaron Dell,8477180,2017-10-07,S.J_2017-10-07,1110.550000,759.0,533.0,37.0,44.64,204.0,26.0,0.951252,7.64,0.412768,0.872549
21,2017-10-14 NYI at S.J,S.J,57.450000,75,48,60.98,52,30,63.41,41,22,65.08,1,2,33.33,2.99,1.42,67.78,31,15,67.39,1,2,33.33,12,4,75,1,1,50.00,19,11,63.33,0,1,0.00,34,22,60.71,0,0,-,2.44,90.91,0.933,1,4,0,0,100.00,22,10,18,55,Aaron Dell,8477180,2017-10-14,S.J_2017-10-14,1073.583333,740.0,519.0,35.0,43.80,202.0,24.0,0.952703,8.80,0.491811,0.881188
22,2017-10-21 S.J at NYI,S.J,58.883333,59,49,54.63,43,33,56.58,29,22,56.86,2,4,33.33,1.83,1.71,51.69,24,16,60,1,4,20.00,6,9,40,0,3,0.00,18,7,72,1,1,50.00,30,21,58.82,1,0,100.00,6.9,81.82,0.887,0,5,0,0,-,20,24,11,64.52,Aaron Dell,8477180,2017-10-21,S.J_2017-10-21,1091.250000,746.0,523.0,35.0,43.84,201.0,24.0,0.953083,8.84,0.486048,0.880597
23,2017-10-28 S.J at BUF,S.J,59.866667,46,57,44.66,36,40,47.37,29,33,46.77,3,2,60.00,2.24,2.32,49.1,19,22,46.34,2,1,66.67,10,9,52.63,2,0,100.00,9,13,40.91,0,1,0.00,25,27,48.08,1,1,50.00,10.34,93.94,1.043,1,3,0,0,100.00,17,24,21,44.74,Aaron Dell,8477180,2017-10-28,S.J_2017-10-28,1092.000000,734.0,512.0,38.0,42.06,193.0,26.0,0.948229,4.06,0.223077,0.865285
24,2017-11-08 T.B at S.J,S.J,20.000000,27,10,72.97,19,10,65.52,13,9,59.09,0,1,0.00,0.75,0.46,62.01,11,3,78.57,0,1,0.00,3,2,60,0,1,0.00,8,1,88.89,0,0,-,14,6,70,0,0,-,0,88.89,0.889,0,1,0,0,-,8,5,3,72.73,Aaron Dell,8477180,2017-11-08,S.J_2017-11-08,1091.866667,731.0,517.0,38.0,42.39,193.0,25.0,0.948016,4.39,0.241238,0.870466
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3,2021-03-01 TOR at EDM,TOR,59.766667,39,56,41.05,33,42,44,22,31,41.51,3,0,100,2.00,2.29,46.64,25,24,51.02,2,0,100,8,8,50,1,0,100,17,16,51.52,1,0,100.00,10,27,27.03,1,0,100.00,13.64,100,1.136,1,3,0,0,100.00,15,14,16,48.39,Michael Hutchinson,8474636,2021-03-01,TOR_2021-03-01,1083.066667,771.0,560.0,59.0,46.67,207.0,32.0,0.923476,-12.33,-0.683060,0.845411
4,2021-03-04 TOR at VAN,TOR,57.500000,51,48,51.52,34,35,49.28,29,28,50.88,1,3,25,2.39,1.53,60.98,29,17,63.04,1,2,33.33,11,6,64.71,1,1,50,18,11,62.07,0,1,0.00,18,30,37.5,0,1,0.00,3.45,89.29,0.927,1,3,0,0,100.00,22,13,11,66.67,Michael Hutchinson,8474636,2021-03-04,TOR_2021-03-04,1084.900000,769.0,562.0,56.0,46.29,205.0,31.0,0.927178,-9.71,-0.537008,0.848780
5,2021-03-14 TOR at OTT,TOR,6.216667,2,11,15.38,2,10,16.67,1,3,25,0,2,0,0.08,0.57,11.68,1,7,12.5,0,2,0,1,5,16.67,0,2,0,0,2,0,0,0,-,1,3,25,0,0,-,0,33.33,0.333,0,1,0,0,-,2,4,1,66.67,Michael Hutchinson,8474636,2021-03-14,TOR_2021-03-14,1077.533333,745.0,548.0,54.0,43.99,194.0,28.0,0.927517,-10.01,-0.557384,0.855670
6,2021-03-29 EDM at TOR,TOR,60.283333,53,43,55.21,45,31,59.21,31,23,57.41,2,3,40,2.20,1.62,57.59,33,20,62.26,2,3,40,12,7,63.16,1,2,33.33,21,13,61.76,1,1,50.00,19,18,51.35,0,0,-,6.45,86.96,0.934,0,4,0,0,-,20,13,19,51.28,Michael Hutchinson,8474636,2021-03-29,TOR_2021-03-29,1070.700000,747.0,546.0,56.0,44.16,195.0,30.0,0.925033,-11.84,-0.663491,0.846154


In [374]:
eg_TOI = eg_df['TOI'].sum()
eg_FA = eg_df['FA'].sum()
eg_GA = eg_df['GA'].sum()
eg_xGA = eg_df['xGA'].sum()
eg_HDCA = eg_df['HDCA'].sum()
eg_HDGA = eg_df['HDGA'].sum()
eg_FenwickSV = (eg_FA - eg_GA) /eg_FA
eg_GSAx = eg_xGA - eg_GA
eg_GSAx60 = (eg_GSAx*60) / eg_TOI
eg_HDCSV = (eg_HDCA - eg_HDGA )/ eg_HDCA

### 2017-2018 Season

In [12]:
primary = get_and_format_nst_team_stats('20172018','5v5', 'n')
pp = get_and_format_nst_team_stats('20172018','pp', 'n')
pk = get_and_format_nst_team_stats('20172018','pk', 'n')

In [13]:
features = merge_team_stats(primary,pp,pk)

features = calculate_team_features(features)

In [15]:
features.tail()

,Game,Team,Unnamed: 2,TOI,CF,CA,CF%,FF,FA,FF%,SF,SA,SF%,GF,GA,GF%,xGF,xGA,xGF%,SCF,SCA,SCF%,HDCF,HDCA,HDCF%,HDSF,HDSA,HDSF%,HDGF,HDGA,HDGF%,HDSH%,HDSV%,MDCF,MDCA,MDCF%,MDSF,MDSA,MDSF%,MDGF,MDGA,MDGF%,MDSH%,MDSV%,LDCF,LDCA,LDCF%,LDSF,LDSA,LDSF%,LDGF,LDGA,LDGF%,LDSH%,LDSV%,SH%,SV%,PDO,Attendance,Date,Game_Number,Team_Key,TOI_pk,xGA_pk,TOI_pp,xGF_pp,sum_rolling20_TOI_5v5,sum_rolling20_FF_5v5,sum_rolling20_FA_5v5,sum_rolling20_GF_5v5,sum_rolling20_GA_5v5,sum_rolling20_xGF_5v5,sum_rolling20_xGA_5v5,sum_rolling20_SF_5v5,last_20_FF%_5v5,last_20_GF%_5v5,last_20_xGF%_5v5,last_20_SH%,sum_rolling20_TOI_pp,sum_rolling20_xGF_pp,last20_pp_TOI_per_game,last20_xGF_per_min_pp,sum_rolling20_TOI_pk,sum_rolling20_xGA_pk,last20_pk_TOI_per_game,last20_xGA_per_min_pk,Last_Game_Date,Days_Since_Last_Game,B2B
2537,"2018-04-07 - Canucks 2, Oilers 3",VAN,Limited ReportFull Report,51.583333,52,54,49.06,44,40,52.38,32,28,53.33,2,0,100.00,2.73,2.99,47.71,28,32,46.67,15,16,48.39,12,9,57.14,2,0,100.0,16.67,100.00,13,16,44.83,6,12,33.33,0,0,NaN,0.0,100.0,22,16,57.89,12,5,70.59,0,0,NaN,0.00,100.00,6.25,100.00,1.063,18347,2018-04-07,82,VAN_2018-04-07,1.516667,0.31,4.966667,0.44,997.166667,676.0,727.0,32.0,40.0,34.26,39.28,480.0,48.182466,44.444444,46.586891,6.666667,69.000000,8.72,3.450000,0.126377,105.366667,9.51,5.268333,0.090256,2018-04-05,2 days,0
2538,"2018-04-07 - Stars 4, Kings 2",DAL,Limited ReportFull Report,50.700000,28,56,33.33,24,44,35.29,16,35,31.37,4,2,66.67,2.15,1.75,55.18,22,21,51.16,9,5,64.29,9,5,64.29,3,1,75.0,33.33,80.00,13,16,44.83,4,7,36.36,1,0,100.0,25.0,100.0,5,32,13.51,2,21,8.70,0,1,0.0,0.00,95.24,25.00,94.29,1.193,18230,2018-04-07,82,DAL_2018-04-07,6.000000,0.14,2.000000,0.17,958.366667,632.0,643.0,27.0,33.0,34.14,32.47,445.0,49.568627,45.000000,51.253566,6.067416,94.066667,11.12,4.703333,0.118214,108.033333,14.14,5.401667,0.130886,2018-04-06,1 days,1
2539,"2018-04-07 - Stars 4, Kings 2",L.A,Limited ReportFull Report,50.700000,56,28,66.67,44,24,64.71,35,16,68.63,2,4,33.33,1.75,2.15,44.82,21,22,48.84,5,9,35.71,5,9,35.71,1,3,25.0,20.00,66.67,16,13,55.17,7,4,63.64,0,1,0.0,0.0,75.0,32,5,86.49,21,2,91.30,1,0,100.0,4.76,100.00,5.71,75.00,0.807,18230,2018-04-07,82,L.A_2018-04-07,2.000000,0.17,6.000000,0.14,982.116667,679.0,688.0,41.0,35.0,32.65,35.32,481.0,49.670812,53.947368,48.035898,8.523909,92.366667,13.04,4.618333,0.141176,94.516667,9.07,4.725833,0.095962,2018-04-05,2 days,0
2540,"2018-04-07 - Wild 6, Sharks 3",MIN,Limited ReportFull Report,53.466667,52,56,48.15,31,37,45.59,23,22,51.11,5,2,71.43,1.84,1.53,54.62,25,21,54.35,14,10,58.33,12,6,66.67,3,2,60.0,25.00,66.67,11,11,50.00,4,4,50.00,1,0,100.0,25.0,100.0,22,27,44.90,7,12,36.84,1,0,100.0,14.29,100.00,21.74,90.91,1.126,17562,2018-04-07,82,MIN_2018-04-07,3.250000,0.28,1.250000,0.00,976.766667,677.0,660.0,37.0,34.0,36.27,30.75,492.0,50.635752,52.112676,54.118174,7.520325,90.283333,11.47,4.514167,0.127044,88.000000,7.56,4.400000,0.085909,2018-04-05,2 days,0
2541,"2018-04-07 - Wild 6, Sharks 3",S.J,Limited ReportFull Report,53.466667,56,52,51.85,37,31,54.41,22,23,48.89,2,5,28.57,1.53,1.84,45.38,21,25,45.65,10,14,41.67,6,12,33.33,2,3,40.0,33.33,75.00,11,11,50.00,4,4,50.00,0,1,0.0,0.0,75.0,27,22,55.10,12,7,63.16,0,1,0.0,0.00,85.71,9.09,78.26,0.874,17562,2018-04-07,82,S.J_2018-04-07,1.250000,0.00,3.250000,0.28,1017.316667,757.0,711.0,49.0,39.0,42.40,36.63,546.0,51.566757,55.681818,53.650512,8.974359,85.300000,12.47,4.265000,0.146190,68.700000,7.74,3.435000,0.112664,2018-04-05,2 days,0


In [18]:
results = get_game_results('2017-10-04', '2018-04-08')

Scraping the schedule between 2017-10-04 and 2018-04-08


In [241]:
results.head()

,game_id,date,venue,home_team,away_team,start_time,home_score,away_score,status,Home_Team_Won,Home_Team_Key,Away_Team_Key
0,2017020001,2017-10-04,Bell MTS Place,WPG,TOR,2017-10-04 23:00:00,2,7,Final,0,WPG_2017-10-04,TOR_2017-10-04
1,2017020002,2017-10-04,PPG Paints Arena,PIT,STL,2017-10-05 00:00:00,4,5,Final,0,PIT_2017-10-04,STL_2017-10-04
2,2017020003,2017-10-04,Rogers Place,EDM,CGY,2017-10-05 02:00:00,3,0,Final,1,EDM_2017-10-04,CGY_2017-10-04
3,2017020004,2017-10-04,SAP Center at San Jose,S.J,PHI,2017-10-05 02:30:00,3,5,Final,0,S.J_2017-10-04,PHI_2017-10-04
4,2017020005,2017-10-05,TD Garden,BOS,NSH,2017-10-05 23:00:00,4,3,Final,1,BOS_2017-10-05,NSH_2017-10-05


In [301]:
df_20172018 = merge_starters_and_features(results, goalie_features_dfB, features, feature_columns, goalie_feature_columns)

In [265]:
df_20172018.tail(30)

,game_id,date,venue,home_team,away_team,start_time,home_score,away_score,status,Home_Team_Won,Home_Team_Key,Away_Team_Key,home_goalie,home_Last_20_FenwickSV%,home_Last_20_GSAx/60,home_Last_20_HDCSV%,away_goalie,away_Last_20_FenwickSV%,away_Last_20_GSAx/60,away_Last_20_HDCSV%,home_Team_Key,home_last_20_FF%_5v5,home_last_20_GF%_5v5,home_last_20_xGF%_5v5,home_last_20_SH%,home_last20_pp_TOI_per_game,home_last20_xGF_per_min_pp,home_last20_pk_TOI_per_game,home_last20_xGA_per_min_pk,home_B2B,away_Team_Key,away_last_20_FF%_5v5,away_last_20_GF%_5v5,away_last_20_xGF%_5v5,away_last_20_SH%,away_last20_pp_TOI_per_game,away_last20_xGF_per_min_pp,away_last20_pk_TOI_per_game,away_last20_xGA_per_min_pk,away_B2B
1241,2017021243,2018-04-05,Wells Fargo Center,PHI,CAR,2018-04-05 23:00:00,4,3,Final,1,PHI_2018-04-05,CAR_2018-04-05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,PHI_2018-04-05,50.909091,48.837209,48.087655,8.267717,5.266667,0.119525,3.953333,0.095742,0,CAR_2018-04-05,54.005525,48.421053,52.547086,8.378871,4.151667,0.114532,3.551667,0.104317,0
1242,2017021244,2018-04-05,Capital One Arena,WSH,NSH,2018-04-05 23:00:00,3,4,Final,0,WSH_2018-04-05,NSH_2018-04-05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,WSH_2018-04-05,49.273256,54.054054,50.321143,8.230453,4.467500,0.120761,5.623333,0.120747,0,NSH_2018-04-05,53.154460,58.227848,52.720371,8.695652,4.895000,0.107865,6.375833,0.115436,0
1243,2017021245,2018-04-05,Nationwide Arena,CBJ,PIT,2018-04-05 23:00:00,4,5,Final,0,CBJ_2018-04-05,PIT_2018-04-05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CBJ_2018-04-05,52.000000,58.823529,51.786834,9.689922,4.414167,0.083821,4.964167,0.116233,0,PIT_2018-04-05,53.641457,53.684211,55.853832,8.994709,4.130833,0.144039,4.470000,0.127405,0
1244,2017021246,2018-04-05,Little Caesars Arena,DET,MTL,2018-04-05 23:30:00,3,4,Final,0,DET_2018-04-05,MTL_2018-04-05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,DET_2018-04-05,49.664929,46.341463,50.476324,8.000000,4.736667,0.100176,5.620833,0.120801,0,MTL_2018-04-05,45.790251,44.776119,45.346777,6.741573,4.677500,0.144842,4.851667,0.153968,0
1245,2017021247,2018-04-05,BB&T Center,FLA,BOS,2018-04-05 23:30:00,3,2,Final,1,FLA_2018-04-05,BOS_2018-04-05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,FLA_2018-04-05,51.016393,57.142857,50.108696,8.421053,5.120833,0.109357,3.725833,0.114739,0,BOS_2018-04-05,54.041916,46.666667,55.812600,6.889764,5.358333,0.144355,5.845000,0.096578,0
1246,2017021248,2018-04-05,Bell MTS Place,WPG,CGY,2018-04-06 00:00:00,2,1,Final,1,WPG_2018-04-05,CGY_2018-04-05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,WPG_2018-04-05,51.310345,55.952381,49.986406,8.867925,4.387500,0.100285,5.377500,0.148210,0,CGY_2018-04-05,58.639456,38.554217,57.543017,5.536332,4.896667,0.156229,5.125000,0.106634,0
1247,2017021249,2018-04-05,Rogers Place,EDM,VGK,2018-04-06 01:00:00,4,3,Final,1,EDM_2018-04-05,VGK_2018-04-05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,EDM_2018-04-05,46.143345,47.252747,44.400141,8.669355,4.353333,0.097167,4.738333,0.097292,0,VGK_2018-04-05,48.794326,49.295775,48.302469,7.128310,4.696667,0.130199,5.084167,0.115555,0
1248,2017021250,2018-04-05,Rogers Arena,VAN,ARI,2018-04-06 02:00:00,4,3,Final,1,VAN_2018-04-05,ARI_2018-04-05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,VAN_2018-04-05,47.619048,43.055556,45.872910,6.485356,3.468333,0.122970,5.270000,0.090797,0,ARI_2018-04-05,49.393291,58.571429,51.904762,8.506224,4.637500,0.111914,4.611667,0.163390,0
1249,2017021251,2018-04-05,STAPLES Center,L.A,MIN,2018-04-06 02:30:00,5,4,Final,1,L.A_2018-04-05,MIN_2018-04-05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,L.A_2018-04-05,49.419448,52.777778,47.335062,7.933194,4.340833,0.140411,4.715833,0.092242,0,MIN_2018-04-05,50.999260,54.285714,55.064483,7.554672,4.490000,0.127394,4.022500,0.085022,1
1250,2017021252,2018-04-05,SAP Center at San Jose,S.J,COL,2018-04-06 02:30:00,4,2,Final,1,S.J_2018-04-05,COL_2018-04-05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,S.J_2018-04-05,51.191287,52.173913,52.808704,8.921933,4.638333,0.142939,3.570833,0.122520,0,COL_2018-04-05,46.320346,55.714286,47.203924,8.387097,5.810000,0.088554,4.540833,0.116168,0


In [302]:

df_20172018.isna().sum()

game_id                          0
date                             0
venue                            0
home_team                        0
away_team                        0
start_time                       0
home_score                       0
away_score                       0
status                           0
Home_Team_Won                    0
Home_Team_Key                    0
Away_Team_Key                    0
home_goalie                      0
home_Last_20_FenwickSV%        126
home_Last_20_GSAx/60           126
home_Last_20_HDCSV%            126
away_goalie                      0
away_Last_20_FenwickSV%        162
away_Last_20_GSAx/60           162
away_Last_20_HDCSV%            162
home_Team_Key                    0
home_last_20_FF%_5v5           329
home_last_20_GF%_5v5           329
home_last_20_xGF%_5v5          329
home_last_20_SH%               329
home_last20_pp_TOI_per_game    329
home_last20_xGF_per_min_pp     329
home_last20_pk_TOI_per_game    329
home_last20_xGA_per_

---
### 2018-2019 Season

In [251]:
primary1819 = get_and_format_nst_team_stats('20182019','5v5', 'n')
pp1819 = get_and_format_nst_team_stats('20182019','pp', 'n')
pk1819 = get_and_format_nst_team_stats('20182019','pk', 'n')

In [252]:
features1819 = merge_team_stats(primary1819,pp1819,pk1819)

In [253]:
features1819 = calculate_team_features(features1819)

In [254]:
results1819 = get_game_results('2018-10-03', '2019-04-06')

Scraping the schedule between 2018-10-03 and 2019-04-06


In [307]:
df_20182019 = merge_starters_and_features(results1819, goalie_features_dfB , features1819, feature_columns, goalie_feature_columns)

In [308]:
df_20182019.tail()

,game_id,date,venue,home_team,away_team,start_time,home_score,away_score,status,Home_Team_Won,Home_Team_Key,Away_Team_Key,home_goalie,home_Last_20_FenwickSV%,home_Last_20_GSAx/60,home_Last_20_HDCSV%,away_goalie,away_Last_20_FenwickSV%,away_Last_20_GSAx/60,away_Last_20_HDCSV%,home_Team_Key,home_last_20_FF%_5v5,home_last_20_GF%_5v5,home_last_20_xGF%_5v5,home_last_20_SH%,home_last20_pp_TOI_per_game,home_last20_xGF_per_min_pp,home_last20_pk_TOI_per_game,home_last20_xGA_per_min_pk,home_B2B,away_Team_Key,away_last_20_FF%_5v5,away_last_20_GF%_5v5,away_last_20_xGF%_5v5,away_last_20_SH%,away_last20_pp_TOI_per_game,away_last20_xGF_per_min_pp,away_last20_pk_TOI_per_game,away_last20_xGA_per_min_pk,away_B2B
1318,2018021267,2019-04-06,American Airlines Center,DAL,MIN,2019-04-07 00:00:00,3,0,Final,1,DAL_2019-04-06,MIN_2019-04-06,Ben Bishop,0.958922,0.689728,0.923077,Alex Stalock,0.927746,-0.614854,0.808219,DAL_2019-04-06,49.125364,54.098361,52.671958,6.470588,4.323333,0.120625,4.574167,0.121880,1.0,MIN_2019-04-06,49.752999,45.161290,52.003938,5.645161,5.189167,0.097318,4.146667,0.082838,0.0
1319,2018021269,2019-04-06,Gila River Arena,ARI,WPG,2019-04-07 02:00:00,2,4,Final,0,ARI_2019-04-06,WPG_2019-04-06,Calvin Pickard,0.908805,-1.222352,0.811429,Connor Hellebuyck,0.937294,-0.151051,0.866071,ARI_2019-04-06,49.018182,56.140351,51.445636,6.543967,5.610000,0.112121,4.285000,0.102917,0.0,WPG_2019-04-06,46.690647,50.000000,43.688785,8.510638,5.008333,0.147854,4.606667,0.113097,0.0
1320,2018021268,2019-04-06,Scotiabank Saddledome,CGY,EDM,2019-04-07 02:00:00,1,3,Final,0,CGY_2019-04-06,EDM_2019-04-06,Mike Smith,0.935829,0.043749,0.890995,Mikko Koskinen,0.929716,-0.480499,0.869565,CGY_2019-04-06,56.752768,61.363636,57.474336,9.782609,5.120000,0.092480,4.489167,0.123408,0.0,EDM_2019-04-06,45.687812,44.943820,44.327894,8.350731,3.325833,0.104635,4.376667,0.118812,0.0
1321,2018021270,2019-04-06,STAPLES Center,L.A,VGK,2019-04-07 02:30:00,5,2,Final,1,L.A_2019-04-06,VGK_2019-04-06,Jonathan Quick,0.907407,-1.504394,0.823834,Marc-Andre Fleury,0.936629,0.047983,0.833333,L.A_2019-04-06,46.001367,39.784946,47.427932,8.078603,4.222500,0.129070,4.032500,0.125728,1.0,VGK_2019-04-06,55.547898,58.823529,56.390790,8.605852,3.981667,0.135245,4.005000,0.103620,0.0
1322,2018021271,2019-04-06,SAP Center at San Jose,S.J,COL,2019-04-07 02:30:00,5,2,Final,1,S.J_2019-04-06,COL_2019-04-06,Martin Jones,0.921196,-0.641724,0.822660,Semyon Varlamov,0.932530,-0.217921,0.861607,S.J_2019-04-06,53.209590,46.590909,53.004873,8.418891,4.920833,0.124776,4.365000,0.118442,0.0,COL_2019-04-06,49.443207,50.000000,49.039005,6.150794,5.243333,0.101271,4.666667,0.136821,0.0


In [310]:
df_20182019.isna().sum()

game_id                          0
date                             0
venue                            0
home_team                        0
away_team                        0
start_time                       0
home_score                       0
away_score                       0
status                           0
Home_Team_Won                    0
Home_Team_Key                    0
Away_Team_Key                    0
home_goalie                      1
home_Last_20_FenwickSV%        119
home_Last_20_GSAx/60           119
home_Last_20_HDCSV%            119
away_goalie                      2
away_Last_20_FenwickSV%        137
away_Last_20_GSAx/60           137
away_Last_20_HDCSV%            137
home_Team_Key                    1
home_last_20_FF%_5v5           328
home_last_20_GF%_5v5           328
home_last_20_xGF%_5v5          328
home_last_20_SH%               328
home_last20_pp_TOI_per_game    328
home_last20_xGF_per_min_pp     328
home_last20_pk_TOI_per_game    328
home_last20_xGA_per_

---
### 2019-2020 Season

In [255]:
primary1920 = get_and_format_nst_team_stats('20192020','5v5', 'n')
pp1920 = get_and_format_nst_team_stats('20192020','pp', 'n')
pk1920 = get_and_format_nst_team_stats('20192020','pk', 'n')

In [256]:
features1920 = merge_team_stats(primary1920,pp1920,pk1920)

In [257]:
features1920 = calculate_team_features(features1920)

In [258]:
results1920 = get_game_results('2019-10-02', '2020-03-12')

Scraping the schedule between 2019-10-02 and 2020-03-12


In [320]:
df_20192020 = merge_starters_and_features(results1920, goalie_features_dfB , features1920, feature_columns, goalie_feature_columns)

In [321]:
df_20192020.isna().sum()

game_id                          0
date                             0
venue                            0
home_team                        0
away_team                        0
start_time                       0
home_score                       0
away_score                       0
status                           0
Home_Team_Won                    0
Home_Team_Key                    0
Away_Team_Key                    0
home_goalie                      1
home_Last_20_FenwickSV%         78
home_Last_20_GSAx/60            78
home_Last_20_HDCSV%             78
away_goalie                      2
away_Last_20_FenwickSV%         82
away_Last_20_GSAx/60            82
away_Last_20_HDCSV%             82
home_Team_Key                    1
home_last_20_FF%_5v5           320
home_last_20_GF%_5v5           320
home_last_20_xGF%_5v5          320
home_last_20_SH%               320
home_last20_pp_TOI_per_game    320
home_last20_xGF_per_min_pp     320
home_last20_pk_TOI_per_game    320
home_last20_xGA_per_

In [344]:
df_20192020[df_20192020['home_Team_Key'].isna()]

,game_id,date,venue,home_team,away_team,start_time,home_score,away_score,status,Home_Team_Won,Home_Team_Key,Away_Team_Key,home_goalie,home_Last_20_FenwickSV%,home_Last_20_GSAx/60,home_Last_20_HDCSV%,away_goalie,away_Last_20_FenwickSV%,away_Last_20_GSAx/60,away_Last_20_HDCSV%,home_Team_Key,home_last_20_FF%_5v5,home_last_20_GF%_5v5,home_last_20_xGF%_5v5,home_last_20_SH%,home_last20_pp_TOI_per_game,home_last20_xGF_per_min_pp,home_last20_pk_TOI_per_game,home_last20_xGA_per_min_pk,home_B2B,away_Team_Key,away_last_20_FF%_5v5,away_last_20_GF%_5v5,away_last_20_xGF%_5v5,away_last_20_SH%,away_last20_pp_TOI_per_game,away_last20_xGF_per_min_pp,away_last20_pk_TOI_per_game,away_last20_xGA_per_min_pk,away_B2B
792,2019120001,2020-01-24,Enterprise Center,AMERICAN ALL-STARS,CANADIAN ALL-STARS,2020-01-25 02:30:00,1,2,Final,0,AMERICAN ALL-STARS_2020-01-24,CANADIAN ALL-STARS_2020-01-24,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


---
### 2020-2021 Season

In [328]:
primary2021 = get_and_format_nst_team_stats('20202021','5v5', 'n')
pp2021 = get_and_format_nst_team_stats('20202021','pp', 'n')
pk2021 = get_and_format_nst_team_stats('20202021','pk', 'n')

In [331]:
pp2021.shape

(1506, 62)

In [332]:
features2021 = merge_team_stats(primary2021,pp2021,pk2021)

In [333]:
features2021 = calculate_team_features(features2021)

In [335]:
features2021.head()

,Game,Team,Unnamed: 2,TOI,CF,CA,CF%,FF,FA,FF%,SF,SA,SF%,GF,GA,GF%,xGF,xGA,xGF%,SCF,SCA,SCF%,HDCF,HDCA,HDCF%,HDSF,HDSA,HDSF%,HDGF,HDGA,HDGF%,HDSH%,HDSV%,MDCF,MDCA,MDCF%,MDSF,MDSA,MDSF%,MDGF,MDGA,MDGF%,MDSH%,MDSV%,LDCF,LDCA,LDCF%,LDSF,LDSA,LDSF%,LDGF,LDGA,LDGF%,LDSH%,LDSV%,SH%,SV%,PDO,Attendance,Date,Game_Number,Team_Key,TOI_pk,xGA_pk,TOI_pp,xGF_pp,sum_rolling20_TOI_5v5,sum_rolling20_FF_5v5,sum_rolling20_FA_5v5,sum_rolling20_GF_5v5,sum_rolling20_GA_5v5,sum_rolling20_xGF_5v5,sum_rolling20_xGA_5v5,sum_rolling20_SF_5v5,last_20_FF%_5v5,last_20_GF%_5v5,last_20_xGF%_5v5,last_20_SH%,sum_rolling20_TOI_pp,sum_rolling20_xGF_pp,last20_pp_TOI_per_game,last20_xGF_per_min_pp,sum_rolling20_TOI_pk,sum_rolling20_xGA_pk,last20_pk_TOI_per_game,last20_xGA_per_min_pk,Last_Game_Date,Days_Since_Last_Game,B2B
0,"2021-01-13 - Penguins 3, Flyers 6",PHI,Limited ReportFull Report,51.933333,38,48,44.19,30,37,44.78,22,30,42.31,4,2,66.67,1.94,1.58,55.04,18,19,48.65,10,7,58.82,8,6,57.14,4,1,80.0,50.00,83.33,8,12,40.0,2,7,22.22,0,1,0.0,0.00,85.71,16,23,41.03,10,15,40.00,0,0,NaN,0.0,100.00,18.18,93.33,1.115,0,2021-01-13,1,PHI_2021-01-13,5.783333,0.28,2.133333,0.28,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT,0
1,"2021-01-13 - Penguins 3, Flyers 6",PIT,Limited ReportFull Report,51.933333,48,38,55.81,37,30,55.22,30,22,57.69,2,4,33.33,1.58,1.94,44.96,19,18,51.35,7,10,41.18,6,8,42.86,1,4,20.0,16.67,50.00,12,8,60.0,7,2,77.78,1,0,100.0,14.29,100.00,23,16,58.97,15,10,60.00,0,0,NaN,0.0,100.00,6.67,81.82,0.885,0,2021-01-13,1,PIT_2021-01-13,2.133333,0.28,5.783333,0.28,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT,0
2,"2021-01-13 - Blackhawks 1, Lightning 5",CHI,Limited ReportFull Report,45.583333,33,34,49.25,23,27,46.00,19,22,46.34,0,3,0.00,1.07,1.79,37.29,19,24,44.19,2,7,22.22,2,7,22.22,0,2,0.0,0.00,71.43,17,17,50.0,12,9,57.14,0,1,0.0,0.00,88.89,8,9,47.06,5,6,45.45,0,0,NaN,0.0,100.00,0.00,86.36,0.864,0,2021-01-13,1,CHI_2021-01-13,7.350000,0.93,6.300000,0.49,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT,0
3,"2021-01-13 - Blackhawks 1, Lightning 5",T.B,Limited ReportFull Report,45.583333,34,33,50.75,27,23,54.00,22,19,53.66,3,0,100.00,1.79,1.07,62.71,24,19,55.81,7,2,77.78,7,2,77.78,2,0,100.0,28.57,100.00,17,17,50.0,9,12,42.86,1,0,100.0,11.11,100.00,9,8,52.94,6,5,54.55,0,0,NaN,0.0,100.00,13.64,100.00,1.136,0,2021-01-13,1,T.B_2021-01-13,6.300000,0.49,7.350000,0.93,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT,0
4,"2021-01-13 - Canadiens 4, Maple Leafs 5",MTL,Limited ReportFull Report,48.600000,47,52,47.47,33,33,50.00,22,20,52.38,2,2,50.00,1.78,1.77,50.06,25,30,45.45,9,6,60.00,5,4,55.56,1,1,50.0,20.00,75.00,16,24,40.0,7,9,43.75,1,0,100.0,14.29,100.00,18,20,47.37,8,7,53.33,0,1,0.0,0.0,85.71,9.09,90.00,0.991,0,2021-01-13,1,MTL_2021-01-13,5.983333,1.64,5.416667,0.88,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT,0


In [336]:
results2021 = get_game_results('2021-01-13', '2021-04-29')

Scraping the schedule between 2021-01-13 and 2021-04-29


In [337]:
results2021.shape

(767, 12)

In [338]:
df_20202021 = merge_starters_and_features(results2021, goalie_features_dfB, features2021, feature_columns, goalie_feature_columns)

In [339]:
df_20202021.isna().sum()

game_id                          0
date                             0
venue                            0
home_team                        0
away_team                        0
start_time                       0
home_score                       0
away_score                       0
status                           0
Home_Team_Won                    0
Home_Team_Key                    0
Away_Team_Key                    0
home_goalie                     10
home_Last_20_FenwickSV%        110
home_Last_20_GSAx/60           110
home_Last_20_HDCSV%            110
away_goalie                     12
away_Last_20_FenwickSV%        115
away_Last_20_GSAx/60           115
away_Last_20_HDCSV%            115
home_Team_Key                    1
home_last_20_FF%_5v5           324
home_last_20_GF%_5v5           324
home_last_20_xGF%_5v5          324
home_last_20_SH%               324
home_last20_pp_TOI_per_game    324
home_last20_xGF_per_min_pp     324
home_last20_pk_TOI_per_game    324
home_last20_xGA_per_

In [340]:
#folling up with NST about this missing game
df_20202021[df_20202021['home_Team_Key'].isna()]

,game_id,date,venue,home_team,away_team,start_time,home_score,away_score,status,Home_Team_Won,Home_Team_Key,Away_Team_Key,home_goalie,home_Last_20_FenwickSV%,home_Last_20_GSAx/60,home_Last_20_HDCSV%,away_goalie,away_Last_20_FenwickSV%,away_Last_20_GSAx/60,away_Last_20_HDCSV%,home_Team_Key,home_last_20_FF%_5v5,home_last_20_GF%_5v5,home_last_20_xGF%_5v5,home_last_20_SH%,home_last20_pp_TOI_per_game,home_last20_xGF_per_min_pp,home_last20_pk_TOI_per_game,home_last20_xGA_per_min_pk,home_B2B,away_Team_Key,away_last_20_FF%_5v5,away_last_20_GF%_5v5,away_last_20_xGF%_5v5,away_last_20_SH%,away_last20_pp_TOI_per_game,away_last20_xGF_per_min_pp,away_last20_pk_TOI_per_game,away_last20_xGA_per_min_pk,away_B2B
659,2020020651,2021-04-11,Nassau Veterans Memorial Coliseum,NYI,NYR,2021-04-11 23:00:00,3,2,Final,1,NYI_2021-04-11,NYR_2021-04-11,Ilya Sorokin,NaN,NaN,NaN,Igor Shesterkin,0.94335,0.061426,0.896714,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


--- 
## Combine and Save

In [ ]:
all_games_rolling20_noSVA = pd.concat([df_20172018, df_20182019, df_20192020, df_20202021])

In [345]:
#impute goalie stats where lack of games causing NaN
all_games_rolling20_noSVA['away_Last_20_FenwickSV%'] = np.where(all_games_rolling20_noSVA['away_Last_20_FenwickSV%'].isna(), ig_FenwickSV,all_games_rolling20_noSVA['away_Last_20_FenwickSV%'])
all_games_rolling20_noSVA['away_Last_20_GSAx/60'] = np.where(all_games_rolling20_noSVA['away_Last_20_GSAx/60'].isna(), ig_GSAx60, all_games_rolling20_noSVA['away_Last_20_GSAx/60'])
all_games_rolling20_noSVA['away_Last_20_HDCSV%'] = np.where(all_games_rolling20_noSVA['away_Last_20_HDCSV%'].isna(), ig_HDCSV, all_games_rolling20_noSVA['away_Last_20_HDCSV%'])
all_games_rolling20_noSVA['home_Last_20_FenwickSV%'] = np.where(all_games_rolling20_noSVA['home_Last_20_FenwickSV%'].isna(), ig_FenwickSV,all_games_rolling20_noSVA['home_Last_20_FenwickSV%'])
all_games_rolling20_noSVA['home_Last_20_GSAx/60'] = np.where(all_games_rolling20_noSVA['home_Last_20_GSAx/60'].isna(), ig_GSAx60, all_games_rolling20_noSVA['home_Last_20_GSAx/60'])
all_games_rolling20_noSVA['home_Last_20_HDCSV%'] = np.where(all_games_rolling20_noSVA['home_Last_20_HDCSV%'].isna(), ig_HDCSV, all_games_rolling20_noSVA['home_Last_20_HDCSV%'])

In [380]:
all_games_rolling20_noSVA.isna().sum()

game_id                           0
date                              0
venue                             0
home_team                         0
away_team                         0
start_time                        0
home_score                        0
away_score                        0
status                            0
Home_Team_Won                     0
Home_Team_Key                     0
Away_Team_Key                     0
home_goalie                      12
home_Last_20_FenwickSV%           0
home_Last_20_GSAx/60              0
home_Last_20_HDCSV%               0
away_goalie                      16
away_Last_20_FenwickSV%           0
away_Last_20_GSAx/60              0
away_Last_20_HDCSV%               0
home_Team_Key                     3
home_last_20_FF%_5v5           1301
home_last_20_GF%_5v5           1301
home_last_20_xGF%_5v5          1301
home_last_20_SH%               1301
home_last20_pp_TOI_per_game    1301
home_last20_xGF_per_min_pp     1301
home_last20_pk_TOI_per_game 

In [381]:
all_games_rolling20_noSVA.to_csv('data/all_games_rolling20_noSVA.csv')

### EDA

In [ ]:
df_20172018['Home_Team_Won'].value_counts(normalize = True)

In [ ]:
df_20172018.columns

In [ ]:
df_20172018[(df_20172018['home_B2B'] == 1) & (df_20172018['away_B2B'] == 0)]['Home_Team_Won'].value_counts(normalize = True)

In [ ]:
df_20172018[(df_20172018['home_B2B'] == 0) & (df_20172018['away_B2B'] == 1)]['Home_Team_Won'].value_counts(normalize = True)

### Quick Model

Evaluating whether rolling 5, 10, 15, 20, or 25 games provides most accurate results. Will use CV results from simple Logistic Regression to Determine

In [383]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import normalize
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.metrics import log_loss
from scipy import stats
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.metrics import roc_auc_score, roc_curve, auc

from sklearn.metrics import plot_confusion_matrix
from sklearn.metrics import confusion_matrix
from sklearn.metrics import plot_roc_curve
from sklearn.metrics import confusion_matrix, plot_confusion_matrix,\
    precision_score, recall_score, accuracy_score, f1_score, log_loss,\
    roc_curve, roc_auc_score, classification_report
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsClassifier, NearestNeighbors
from sklearn.tree import DecisionTreeClassifier
from collections import Counter
from imblearn.under_sampling import TomekLinks 
from sklearn.preprocessing import StandardScaler, OneHotEncoder, MinMaxScaler

from sklearn.pipeline import make_pipeline

#### 20 Game Rolling

In [385]:
model_df = all_games_rolling20_noSVA.dropna()

In [386]:
model_df.isna().sum()

game_id                        0
date                           0
venue                          0
home_team                      0
away_team                      0
start_time                     0
home_score                     0
away_score                     0
status                         0
Home_Team_Won                  0
Home_Team_Key                  0
Away_Team_Key                  0
home_goalie                    0
home_Last_20_FenwickSV%        0
home_Last_20_GSAx/60           0
home_Last_20_HDCSV%            0
away_goalie                    0
away_Last_20_FenwickSV%        0
away_Last_20_GSAx/60           0
away_Last_20_HDCSV%            0
home_Team_Key                  0
home_last_20_FF%_5v5           0
home_last_20_GF%_5v5           0
home_last_20_xGF%_5v5          0
home_last_20_SH%               0
home_last20_pp_TOI_per_game    0
home_last20_xGF_per_min_pp     0
home_last20_pk_TOI_per_game    0
home_last20_xGA_per_min_pk     0
home_B2B                       0
away_Team_

In [387]:
model_df.columns

Index(['game_id', 'date', 'venue', 'home_team', 'away_team', 'start_time',
       'home_score', 'away_score', 'status', 'Home_Team_Won', 'Home_Team_Key',
       'Away_Team_Key', 'home_goalie', 'home_Last_20_FenwickSV%',
       'home_Last_20_GSAx/60', 'home_Last_20_HDCSV%', 'away_goalie',
       'away_Last_20_FenwickSV%', 'away_Last_20_GSAx/60',
       'away_Last_20_HDCSV%', 'home_Team_Key', 'home_last_20_FF%_5v5',
       'home_last_20_GF%_5v5', 'home_last_20_xGF%_5v5', 'home_last_20_SH%',
       'home_last20_pp_TOI_per_game', 'home_last20_xGF_per_min_pp',
       'home_last20_pk_TOI_per_game', 'home_last20_xGA_per_min_pk', 'home_B2B',
       'away_Team_Key', 'away_last_20_FF%_5v5', 'away_last_20_GF%_5v5',
       'away_last_20_xGF%_5v5', 'away_last_20_SH%',
       'away_last20_pp_TOI_per_game', 'away_last20_xGF_per_min_pp',
       'away_last20_pk_TOI_per_game', 'away_last20_xGA_per_min_pk',
       'away_B2B'],
      dtype='object')

In [388]:
features = ['home_Last_20_FenwickSV%',
       'home_Last_20_GSAx/60', 'home_Last_20_HDCSV%',
       'away_Last_20_FenwickSV%', 'away_Last_20_GSAx/60',
       'away_Last_20_HDCSV%', 'home_last_20_FF%_5v5',
       'home_last_20_GF%_5v5', 'home_last_20_xGF%_5v5', 'home_last_20_SH%',
       'home_last20_pp_TOI_per_game', 'home_last20_xGF_per_min_pp',
       'home_last20_pk_TOI_per_game', 'home_last20_xGA_per_min_pk', 'home_B2B', 'away_last_20_FF%_5v5', 'away_last_20_GF%_5v5',
       'away_last_20_xGF%_5v5', 'away_last_20_SH%',
       'away_last20_pp_TOI_per_game', 'away_last20_xGF_per_min_pp',
       'away_last20_pk_TOI_per_game', 'away_last20_xGA_per_min_pk',
       'away_B2B']

In [411]:
X = model_df[features]
y = model_df['Home_Team_Won']

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=2021)

pipeline = make_pipeline(StandardScaler(), LogisticRegression(random_state=2021,  max_iter=1000))

accuracy_20 = cross_val_score(pipeline, X_train, y_train, cv=5, scoring = 'accuracy').mean()
accuracy_20

0.5946502057613168

In [403]:
model_df.shape

(3241, 40)

### 15 Game Rolling

In [397]:
#17-18
features_b = merge_team_stats(primary,pp,pk)
features_b = calculate_team_features(features_b, 15)
df_20172018_b = merge_starters_and_features(results, goalie_features_dfB, features_b, feature_columns, goalie_feature_columns)

In [398]:
#18-19
features_1819_b = merge_team_stats(primary1819,pp1819,pk1819)
features_1819_b = calculate_team_features(features_1819_b, 15)
df_20182019_b = merge_starters_and_features(results1819, goalie_features_dfB, features_1819_b, feature_columns, goalie_feature_columns)

In [402]:
#19-20
features_1920_b = merge_team_stats(primary1920,pp1920,pk1920)
features_1920_b = calculate_team_features(features_1920_b, 15)
df_20192020_b = merge_starters_and_features(results1920, goalie_features_dfB, features_1920_b, feature_columns, goalie_feature_columns)

In [404]:
#20-21
features_2021_b = merge_team_stats(primary2021,pp2021,pk2021)
features_2021_b = calculate_team_features(features_2021_b, 15)
df_20202021_b = merge_starters_and_features(results2021, goalie_features_dfB, features_2021_b, feature_columns, goalie_feature_columns)

In [405]:
all_games_rolling15_noSVA = pd.concat([df_20172018_b, df_20182019_b, df_20192020_b, df_20202021_b])

#impute goalie stats where lack of games causing NaN
all_games_rolling15_noSVA['away_Last_20_FenwickSV%'] = np.where(all_games_rolling15_noSVA['away_Last_20_FenwickSV%'].isna(), ig_FenwickSV,all_games_rolling15_noSVA['away_Last_20_FenwickSV%'])
all_games_rolling15_noSVA['away_Last_20_GSAx/60'] = np.where(all_games_rolling15_noSVA['away_Last_20_GSAx/60'].isna(), ig_GSAx60, all_games_rolling15_noSVA['away_Last_20_GSAx/60'])
all_games_rolling15_noSVA['away_Last_20_HDCSV%'] = np.where(all_games_rolling15_noSVA['away_Last_20_HDCSV%'].isna(), ig_HDCSV, all_games_rolling15_noSVA['away_Last_20_HDCSV%'])
all_games_rolling15_noSVA['home_Last_20_FenwickSV%'] = np.where(all_games_rolling15_noSVA['home_Last_20_FenwickSV%'].isna(), ig_FenwickSV,all_games_rolling15_noSVA['home_Last_20_FenwickSV%'])
all_games_rolling15_noSVA['home_Last_20_GSAx/60'] = np.where(all_games_rolling15_noSVA['home_Last_20_GSAx/60'].isna(), ig_GSAx60, all_games_rolling15_noSVA['home_Last_20_GSAx/60'])
all_games_rolling15_noSVA['home_Last_20_HDCSV%'] = np.where(all_games_rolling15_noSVA['home_Last_20_HDCSV%'].isna(), ig_HDCSV, all_games_rolling15_noSVA['home_Last_20_HDCSV%'])

In [419]:
model_df_b = all_games_rolling15_noSVA.dropna()

X = model_df_b[features]
y = model_df_b['Home_Team_Won']

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=2021)

pipeline = make_pipeline(StandardScaler(), LogisticRegression(random_state=2021,  max_iter=1000))

accuracy_15 = cross_val_score(pipeline, X_train, y_train, cv=5, scoring = 'accuracy').mean()
accuracy_15

0.5947565543071162

### 10 Game Rolling

In [412]:
#17-18
features_c = merge_team_stats(primary,pp,pk)
features_c = calculate_team_features(features_c, 10)
df_20172018_c = merge_starters_and_features(results, goalie_features_dfB, features_c, feature_columns, goalie_feature_columns)

#18-19
features_1819_c = merge_team_stats(primary1819,pp1819,pk1819)
features_1819_c = calculate_team_features(features_1819_c, 10)
df_20182019_c = merge_starters_and_features(results1819, goalie_features_dfB, features_1819_c, feature_columns, goalie_feature_columns)

#19-20
features_1920_c = merge_team_stats(primary1920,pp1920,pk1920)
features_1920_c = calculate_team_features(features_1920_c, 10)
df_20192020_c = merge_starters_and_features(results1920, goalie_features_dfB, features_1920_c, feature_columns, goalie_feature_columns)

#20-21
features_2021_c = merge_team_stats(primary2021,pp2021,pk2021)
features_2021_c = calculate_team_features(features_2021_c, 10)
df_20202021_c = merge_starters_and_features(results2021, goalie_features_dfB, features_2021_c, feature_columns, goalie_feature_columns)

In [416]:
all_games_rolling10_noSVA = pd.concat([df_20172018_c, df_20182019_c, df_20192020_c, df_20202021_c])

#impute goalie stats where lack of games causing NaN
all_games_rolling10_noSVA['away_Last_20_FenwickSV%'] = np.where(all_games_rolling10_noSVA['away_Last_20_FenwickSV%'].isna(), ig_FenwickSV,all_games_rolling10_noSVA['away_Last_20_FenwickSV%'])
all_games_rolling10_noSVA['away_Last_20_GSAx/60'] = np.where(all_games_rolling10_noSVA['away_Last_20_GSAx/60'].isna(), ig_GSAx60, all_games_rolling10_noSVA['away_Last_20_GSAx/60'])
all_games_rolling10_noSVA['away_Last_20_HDCSV%'] = np.where(all_games_rolling10_noSVA['away_Last_20_HDCSV%'].isna(), ig_HDCSV, all_games_rolling10_noSVA['away_Last_20_HDCSV%'])
all_games_rolling10_noSVA['home_Last_20_FenwickSV%'] = np.where(all_games_rolling10_noSVA['home_Last_20_FenwickSV%'].isna(), ig_FenwickSV,all_games_rolling10_noSVA['home_Last_20_FenwickSV%'])
all_games_rolling10_noSVA['home_Last_20_GSAx/60'] = np.where(all_games_rolling10_noSVA['home_Last_20_GSAx/60'].isna(), ig_GSAx60, all_games_rolling10_noSVA['home_Last_20_GSAx/60'])
all_games_rolling10_noSVA['home_Last_20_HDCSV%'] = np.where(all_games_rolling10_noSVA['home_Last_20_HDCSV%'].isna(), ig_HDCSV, all_games_rolling10_noSVA['home_Last_20_HDCSV%'])

In [417]:
model_df_c = all_games_rolling10_noSVA.dropna()

X = model_df_c[features]
y = model_df_c['Home_Team_Won']

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=2021)

pipeline = make_pipeline(StandardScaler(), LogisticRegression(random_state=2021,  max_iter=1000))

accuracy_10 = cross_val_score(pipeline, X_train, y_train, cv=5, scoring = 'accuracy').mean()
accuracy_10

0.557659457834521

### 5 Game Rolling